<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/BirminghamMethod_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *the Birmingham Method*

> [1] Parker DJ, Broadbent CJ, Fowles P, Hawkesworth MR, McNeil P. Positron emission particle tracking-a technique for studying flow within engineering equipment. Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment. 1993 Mar 10;326(3):592-607.

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the Birmingham Method<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code. 

In [ ]:
# First install the `pept` library using pip, Python's package manager
!pip install git+https://github.com/uob-positron-imaging-centre/pept.git

  Cloning https://github.com/uob-positron-imaging-centre/pept.git to /tmp/pip-req-build-qe7u5wvt
  Running command git clone -q https://github.com/uob-positron-imaging-centre/pept.git /tmp/pip-req-build-qe7u5wvt
  Created wheel for pept: filename=pept-0.3.0-cp37-cp37m-linux_x86_64.whl size=4131239 sha256=daa743e5e5e6ae77cb13ce26f9f27a8d755a5eb834a6a71a6d8cd1d7871186f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-gdicg0c_/wheels/3e/b2/c9/8890c6e267a3f17365896ef7d1dec1f322d29677da9e80cf0e
Successfully built pept


# 2. The Birmingham Method

## 2.1. Read in Line of Response Data

In [ ]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 50,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       [2.000e-01, 0.000e+00, 2.962e+02, 6.000e+02, 4.525e+02],
       [2.000e-01, 0.000e+00, 1.151e+02, 6.000e+02, 3.534e+02],
       [2.000e-01, 0.000e+00, 1.322e+02, 6.000e+02, 2.661e+02],
       [2.000e-01, 0.000e+00, 3.735e+02, 6.000e+02, 1.310e+02],
       [3.000e-01, 0.000e+00, 1.115e+02, 6.000e+02, 3.534e+02],
       [4.000e-01, 0.000e+00, 2.094e+02, 6.000e+02, 2.808e+02],
       [4.000e-01, 0.000e+00, 2.749e+02, 6.000e+02, 3.062e+02],
       [5.000e-01, 0.000e+00, 2.389e+02, 6.000e+02, 1.681e+02],
       [6.000e-01, 0.000e+00, 4.283e+02, 6.000e+02, 7.020e+01],
       [6.000e-01, 0.000e+00, 1.333e+02, 6.000e+02, 3.499e+02],
       [6.000e-01, 0.000e+00, 3.050e+02, 6.000e+02, 1.032e+02],
       [6.000e-01, 0.000e+00, 9.200e+01, 6.000e+02, 3.929e+02],
       [6.000e-01, 0.000e+00, 1.859e+02,

In [ ]:
from pept.visualisation import PlotlyGrapher2D

grapher = PlotlyGrapher2D()
grapher.add_lines(lors)
grapher.show()

## 2.2. Find Minimum Distance Point

In [ ]:
# Find the minimum distance point (MDP) to all LoRs by minimising the sum of distances to all lines
from scipy.optimize import minimize

def distances(point, lines):
    d1 = np.abs((lines[:, 3] - lines[:, 1]) * (lines[:, 2] - point[1]) - 
                (lines[:, 4] - lines[:, 2]) * (lines[:, 1] - point[0]))
    d2 = np.sqrt((lines[:, 3] - lines[:, 1]) ** 2 + (lines[:, 4] - lines[:, 2]) ** 2)
    return d1 / d2


def sum_distances(point, lines):
    return np.sum(distances(point, lines))

p0 = [250, 250]     # initial guess
mdp = minimize(sum_distances, p0, lors).x

# Insert MDP timestamp at column 0
mdp = np.insert(mdp, 0, lors[:, 0].mean())
mdp

array([  0.938     , 316.15441987, 243.17592997])

In [ ]:
# Plot all LoRs and the minimum distance point (MDP)
grapher = PlotlyGrapher2D()

grapher.add_lines(lors)
grapher.add_points([mdp])

grapher.show()

## 2.3. Remove the Farthest Lines of Response

In [ ]:
# Remove the 10 farthest LoRs from the MDP to eliminate scattered LoRs, then recompute the MDP

# Compute all distances from the MDP to the LoRs, then find the *indices* of the sorted distances.
# The last 10 elements in `sorted_dists` are the indices of the farthest LoRs
dists = distances(mdp[1:], lors)        # Ignore the time column (index 0)
sorted_dists = np.argsort(dists)

scattered_lors = lors[sorted_dists[-10:]]
good_lors = lors[sorted_dists[:-10]]

# Recompute the MDP only from the retained ("good") LoRs
p0 = [250, 250]
new_mdp = minimize(sum_distances, p0, good_lors).x

# Insert MDP timestamp at column 0
new_mdp = np.insert(new_mdp, 0, good_lors[:, 0].mean())

grapher = PlotlyGrapher2D()

grapher.add_lines(good_lors, color = "green")
grapher.add_lines(scattered_lors, color = "black")

grapher.add_points([mdp], color = "red")
grapher.add_points([new_mdp], color = "blue")

grapher.show()

## 2.4. Iteratively Remove the Farthest LoRs and Recompute MDP

In [ ]:
# The Birmingham method works by iteratively removing scattered LoRs:
#
#     1. Compute MDP
#     2. Remove `nlors_remove` LoRs that are farthest away from the MDP
#     3. Re-compute MDP
#
# Until `nlors_target` LoRs remain

nlors_remove = 5
nlors_target = 10

# Indices of "good" / unscattered LoRs
good_indices = np.arange(len(lors))

# Find the initial minimum distance point (MDP)
p0 = [250, 250]
cur_mdp = minimize(sum_distances, p0, lors[good_indices]).x

# Iteratively remove `nlors_remove` farthest LoRs at a time and recompute the MDP until
# `nlors_target` LoRs remain
while len(good_indices) > nlors_target:
    dists = distances(cur_mdp, lors[good_indices])              # Distances from MDP to all good LoRs
    sorted_dists = np.argsort(dists)                            # Sorted indices of distances
    good_indices = good_indices[sorted_dists[:-nlors_remove]]   # Remove `nlors_remove` good LoR indices

    cur_mdp = minimize(sum_distances, cur_mdp, lors[good_indices]).x

# Insert the tracer position's timestamp at column 0
position = np.insert(cur_mdp, 0, lors[good_indices, 0].mean())

In [ ]:
# Plot the remaining good LoRs and the final tracer location
grapher = PlotlyGrapher2D()

grapher.add_lines(lors[good_indices])
grapher.add_points([position])

grapher.show()

# 3. Complete Birmingham Method Code

In [ ]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 1000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       ...,
       [3.560e+01, 0.000e+00, 8.440e+01, 6.000e+02, 2.384e+02],
       [3.570e+01, 0.000e+00, 5.463e+02, 6.000e+02, 7.670e+01],
       [3.570e+01, 0.000e+00, 5.334e+02, 6.000e+02, 7.020e+01]])

In [ ]:
# Use the Birmingham Method to track moving tracer
from scipy.optimize import minimize

def distances(point, lines):
    d1 = np.abs((lines[:, 3] - lines[:, 1]) * (lines[:, 2] - point[1]) - 
                (lines[:, 4] - lines[:, 2]) * (lines[:, 1] - point[0]))
    d2 = np.sqrt((lines[:, 3] - lines[:, 1]) ** 2 + (lines[:, 4] - lines[:, 2]) ** 2)
    return d1 / d2


def sum_distances(point, lines):
    return np.sum(distances(point, lines))


nlors_remove = 5
nlors_target = 10

sample_start = 0
sample_size = 100

positions = []

while sample_start < len(lors):
    sample = lors[sample_start:sample_start + sample_size]

    # Indices of "good" / unscattered LoRs
    good_indices = np.arange(len(sample))

    # Find the initial minimum distance point (MDP)
    p0 = [250, 250]
    cur_mdp = minimize(sum_distances, p0, sample[good_indices]).x

    # Iteratively remove `nlors_remove` farthest LoRs at a time and recompute the MDP until
    # `nlors_target` LoRs remain
    while len(good_indices) > nlors_target:
        dists = distances(cur_mdp, sample[good_indices])            # Distances from MDP to all good LoRs
        sorted_dists = np.argsort(dists)                            # Sorted indices of distances
        good_indices = good_indices[sorted_dists[:-nlors_remove]]   # Remove `nlors_remove` good LoR indices

        cur_mdp = minimize(sum_distances, cur_mdp, sample[good_indices]).x

    # Insert the tracer position's timestamp at column 0
    position = np.insert(cur_mdp, 0, lors[good_indices, 0].mean())
    positions.append(position)

    sample_start += sample_size

positions = np.array(positions)
positions

array([[  1.56      , 320.22868672, 242.28747266],
       [  1.93      , 315.15295696, 244.24180824],
       [  1.72      , 315.03283677, 243.06066528],
       [  1.82      , 319.45421677, 243.83979282],
       [  1.79      , 312.96935594, 244.93927179],
       [  2.05      , 318.76051243, 244.1492841 ],
       [  1.86      , 313.61590891, 243.68524273],
       [  2.67      , 317.81042461, 244.03525861],
       [  2.26      , 317.70296597, 245.05636656],
       [  2.36      , 314.39909502, 245.51651433]])

In [ ]:
grapher = PlotlyGrapher2D(cols = 2)

grapher.add_lines(lors[:500])
grapher.add_points(positions, col = 2)

grapher.show()